In [1]:
import pickle
import re

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Import Dataset

In [2]:
dataset = pd.read_csv('data/train.csv')
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Preprocessing

## Preprocessing functions

In [3]:
def drop_ticket(dataset):
    dataset = dataset.drop('Ticket', axis=1)
    return dataset

In [4]:
def get_all_family_members(dataset):
    dataset['FamilyMembers'] = dataset['SibSp'] + dataset['Parch']
    return dataset

In [5]:
def clean_nans(dataset):
    dataset['Cabin'] = dataset['Cabin'].map(lambda x: x[0] if isinstance(x, str) else 'Z')
    dataset['Embarked'] = dataset['Embarked'].fillna('None')
    dataset['Age'] = dataset['Age'].fillna(-100)
    
    return dataset

In [6]:
def extract_title(name):
    match = re.search(r',\s*([^\.]*)\.', name)
    if match:
        return match.group(1).strip()
    return ""

In [7]:
def transform_name(name):
    maiden_name_match = re.search(r'\(([^()]*\b\w+)\)', name)
    last_name_match = re.match(r'([^,]+),', name)
    title_match = re.search(r',\s*([^\.]*)\.', name)

    last_name = last_name_match.group(1).strip()
    
    if title_match:
        title = title_match.group(1).strip()
        if title == 'Miss' or title == 'Mlle':
            title = 'Ms'
        elif title == 'Mme':
            title = 'Mrs'
        
    if maiden_name_match:
        content_inside_parentheses = maiden_name_match.group(1)
        maiden_name = content_inside_parentheses.split()[-1]
    else:
        maiden_name = last_name
    

    return title, maiden_name, last_name

In [8]:
def drop_full_name(dataset):
    dataset = dataset.drop('Name', axis=1)
    return dataset

In [9]:
def one_hot_encode(dataset):
    one_hot_sex = pd.get_dummies(dataset['Sex'])
    dataset = dataset.drop('Sex', axis=1)
    
    one_hot_cabin = pd.get_dummies(dataset['Cabin'])
    dataset = dataset.drop('Cabin', axis=1)
    
    one_hot_embarked = pd.get_dummies(dataset['Embarked'])
    dataset = dataset.drop('Embarked', axis=1)
    
    one_hot_title = pd.get_dummies(dataset['Title'])
    dataset = dataset.drop('Title', axis=1)
    
    one_hot_maidenname = pd.get_dummies(dataset['MaidenName'])
    dataset = dataset.drop('MaidenName', axis=1)
    
    one_hot_lastname = pd.get_dummies(dataset['LastName'])
    dataset = dataset.drop('LastName', axis=1)

    return pd.concat([dataset, one_hot_sex, one_hot_cabin, one_hot_embarked, one_hot_title, one_hot_maidenname, one_hot_lastname], axis=1)

In [10]:
def split_dataset(dataset):
    X = dataset.drop('Survived', axis=1)
    y = dataset['Survived']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    scaler = StandardScaler()
    #X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    y_train = y_train.values
    y_test = y_test.values
    #X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)
    
    return X_train, X_test, y_train, y_test

## Calling Preprocessing Functions

In [11]:
dataset = drop_ticket(dataset)

In [12]:
dataset = get_all_family_members(dataset)

In [13]:
dataset = clean_nans(dataset)

In [14]:
dataset[['Title', 'MaidenName', 'LastName']] = dataset['Name'].apply(lambda x: pd.Series(transform_name(x)))

In [15]:
dataset = drop_full_name(dataset)

In [16]:
dataset = one_hot_encode(dataset)

# Save New Dataset

In [17]:
X_train, X_test, y_train, y_test = split_dataset(dataset)

In [18]:
with open('data/X_train.pkl', 'wb') as f:
    pickle.dump(X_train, f)
with open('data/X_test.pkl', 'wb') as f:
    pickle.dump(X_test, f)
with open('data/y_train.pkl', 'wb') as f:
    pickle.dump(y_train, f)
with open('data/y_test.pkl', 'wb') as f:
    pickle.dump(y_test, f)